In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [10]:
# Load data
df = pd.read_csv("space_mountain_with_holiday_weather_lag_suite.csv")

# Parse date
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
df['Datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'])
df.sort_values('Datetime', inplace=True)

# One-hot encode Day of Week
df = pd.get_dummies(df, columns=['Day of Week'], drop_first=True)

# Drop unused columns
df.drop(columns=['Date', 'Time', 'Datetime'], inplace=True)

# Scale all features
scaler = StandardScaler()
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns=df.columns)

In [ ]:
SEQUENCE_LENGTH = 12
X, y = [], []
features = scaled_df.drop(columns=['Wait Time']).values
wait_values = scaled_df['Wait Time'].values

for i in range(SEQUENCE_LENGTH, len(scaled_df)):
    X.append(features[i-SEQUENCE_LENGTH:i])
    y.append(wait_values[i])

X = np.array(X)
y = np.array(y)

# Train/test split
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [12]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

/Users/jeremypogue/miniforge3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        20,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,801 (81.25 KB)

 Trainable params: 20,801 (81.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7984 - val_loss: 0.5729
Epoch 2/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.7193 - val_loss: 0.5908
Epoch 3/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7009 - val_loss: 0.5606
Epoch 4/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7222 - val_loss: 0.5907
Epoch 5/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6892 - val_loss: 0.5754
Epoch 6/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6778 - val_loss: 0.5797
Epoch 7/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6752 - val_loss: 0.5918
Epoch 8/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6676 - val_loss: 0.5793
Epoch 9/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6549 - val_loss: 0.5584
Epoch 10/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6679 - val_loss: 0.5872
Epoch 11/20
1085/1085 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6562 - val_loss: 0.6070
Epoch 12/20
1085/1085 ━━━━━━━━

In [13]:
y_pred = model.predict(X_test)

# Unscale predictions and actuals to get real RMSE in minutes
wait_scaler = StandardScaler()
wait_scaler.fit(df[['Wait Time']])  # Fit again just on original column

y_pred_unscaled = wait_scaler.inverse_transform(y_pred)
y_test_unscaled = wait_scaler.inverse_transform(y_test.reshape(-1, 1))

real_rmse = np.sqrt(mean_squared_error(y_test_unscaled, y_pred_unscaled))
print(f"Unscaled RMSE (in minutes): {real_rmse:.2f}")


302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step
Unscaled RMSE (in minutes): 18.44
